In [3]:
## Librería transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
## librerías colab_utils y librosa
!pip install git+https://github.com/ricardodeazambuja/colab_utils.git
!pip install librosa==0.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ricardodeazambuja/colab_utils.git to /tmp/pip-req-build-6gdmg1ys
  Running command git clone --filter=blob:none --quiet https://github.com/ricardodeazambuja/colab_utils.git /tmp/pip-req-build-6gdmg1ys
  Resolved https://github.com/ricardodeazambuja/colab_utils.git to commit be8136f1c8c3494402a9530d23577deb74bfd631
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from colab_utils import getAudio
## librerías para el procesamiento del audio
import librosa
import numpy as np
## pytorch para la conversión de voz a texto
import torch

In [6]:
w2v2_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
w2v2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
## método getAudio de la librería colab_utils.. para capturar el audio desde nuestro micrófono.
audio, sr = getAudio()


In [8]:
# Representar cada muestra en formato punto flotante con 32 bits
audio_float = audio.astype(np.float32)

# Y cambiar la frecuencia de muestreo a 16 KHz
audio_16k = librosa.resample(audio_float, sr, 16000)


In [9]:
# conversión de voz a texto
entrada = w2v2_processor(audio_16k, sampling_rate=16000, return_tensors="pt").input_values

In [10]:
## arreglo de 203 x 32
probabilidades = w2v2(entrada).logits

In [11]:
predicciones = torch.argmax(probabilidades, dim=-1)

In [12]:
## pasa de una representación numérica a una secuencia de caracteres.
transcripcion = w2v2_processor.decode(predicciones[0])

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")

In [14]:
entradaBlender = tokenizer([transcripcion], return_tensors='pt')

In [15]:
from transformers import AutoModelForSeq2SeqLM
blender = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

In [16]:
ids_respuesta = blender.generate(**entradaBlender)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (60) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [17]:
print(ids_respuesta)

tensor([[   1, 2219,  304,  957, 1534,  306,  397, 2154,   43, 2159,   64,   38,
          452,  341,  271, 1085,  341, 3035,   20,   83,  305, 2415,  279, 4332,
          322,   21,    2]])


In [18]:
respuesta = tokenizer.batch_decode(ids_respuesta)

In [19]:
print(respuesta)

["<s> Have you ever heard of MONDANY? It's the world's third-largest mammal.</s>"]


In [20]:
respuesta = respuesta[0].replace('<s>','').replace('</s>','')

In [21]:
NFRASES = 6
nfrase = 1

while nfrase <= NFRASES:
  input()     # Esperar a pulsar tecla para iniciar grabación
  
  # Capturar audio y llevarlo a 16 KHz
  audio, sr = getAudio()
  audio_float = audio.astype(np.float32)
  audio_16k = librosa.resample(audio_float, sr, 16000)

  # Voz a texto con wav2vec2
  entrada = w2v2_processor(audio_16k, sampling_rate=16000, return_tensors="pt").input_values
  probabilidades = w2v2(entrada).logits
  predicciones = torch.argmax(probabilidades, dim=-1)
  frase = w2v2_processor.decode(predicciones[0])
  
  # Imprimir transcripción voz a texto
  print(f'-> MIGUEL: {frase}')

  # BlenderBot
  entradaBlender = tokenizer([frase], return_tensors='pt')
  ids_respuesta = blender.generate(**entradaBlender)
  respuesta = tokenizer.batch_decode(ids_respuesta)
  respuesta = respuesta[0].replace('<s>','').replace('</s>','')
  print(f'-> BLENDERBOT: {respuesta}')

  nfrase += 1

-> MIGUEL: CEEL


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (60) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


-> BLENDERBOT:  I love the smell of celery.  It's one of my favorite fruits.


-> MIGUEL: HELO HOW ARE YOU


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (60) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


-> BLENDERBOT:  I'm doing well, how are you?  I just got back from a long day of work.


KeyboardInterrupt: ignored